In [1]:
import nltk, random,json, pickle
import requests
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import os
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
lemmatizer=WordNetLemmatizer()
context={}

In [3]:
class Testing:
    
    def __init__(self):
        self.intents= json.loads(open("intents.json").read())
        
        data= pickle.load(open("training_data","rb"))
        self.words= data["words"]
        self.classes= data["classes"]
        self.model= load_model("chatbot_model.h5")
        
        self.ERROR_THRESHOLD=0.5
        self.ignore_words= list("!@#$%^&*?")
        
        
    def clean_up_sentence(self, sentence):
        
        sentence_words= word_tokenize(sentence.lower())
        
        sentence_words= list(map(lemmatizer.lematizer, sentence_words))
        sentence_words= list(filter(lambda x:x not in self.ignore_words, sentence_word))
        return set(sentence_words)
    
    
    def wordvector(self, sentence):
        cv= CountVectorizer(tokenizer= lambda txt: txt.split())
        sentence_words=" ".join(self.clean_up_sentence(sentence))
        words=" ".join(self.words)
        
        vectorize= cv.fit([words])
        word_vector= vectorize([sentence_words]).toarray().tolist()[0]
        return np.array(word_vector)
    
    
    def classify(self, sentence):
        results= self.model.predict(np.array([self.wordvector(sentence)]))[0]
        results= list(map(lambda x:[x[0], x[1]], enumerate(results)))
        results= list(filter(lambda x:x[1] > self.ERROR_THRESHOLD, results))
        
        results.sort(key=lambda x:x[1], reverse=True)
        return_list=[]
        
        for i in results:
            return_list.append((self.classes[i[0]], str(i[1])))
        return return_list
    
    
    def results(self, sentence, userID):
        if sentence.isdecimal():
            if context[userID]== "historydetails":
                return self.classify("ordernumber")
        return self.classify(sentence)
    
    def response(self, sentence, uerID="DianaStube"):
        results= self.results(sentence, userID)
        print(sentence, results)
        
        ans=""
        if results:
            while results:
                for i in self.intents["intents"]:
                    if i["tag"] == results[0][0]:
                        if "set" in i and not "filter" in i:
                            context[userID]= i["set"]
                        if not "filter" in i:
                            ans= random.choice(i["responses"])
                            print("Query:", sentence)
                            print("Bot:", ans)
                        if userID in context and "filter" in i and i["filter"]== context[userID]:
                            if "set" in i:
                                context[userID]= i["set"]
                            ans= random.choice(i["responses"])
                            
                results.pop(0)   
        return ans if ans!="" else "Sorry ! i don't know about"  